In [1]:
import os, sys
sys.path.append('ext/deepFibreTracking/')

import numpy as np
import random
import gym
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import envs.RLtractEnvironment as RLTe
from src.data import HCPDataContainer, ISMRMDataContainer

# Initialise environment

In [2]:
RLenv = RLTe.RLtractEnvironment(device = 'cpu', grid_dim=[3,3,3])

Loading precomputed streamlines (data/HCP307200_DTI_smallSet.vtk) for ID 100307
Reset to streamline 3/5


In [3]:
nextState, rewardNextState, done = RLenv.step(10)
print("Reward: %.3f" % (rewardNextState))

Reward: -0.007


In [ ]:
nextState.getValue()

# Tracker

In [ ]:
from src.data import HCPDataContainer, ISMRMDataContainer
from src.data.postprocessing import res100
from src.tracker import DTITracker, CSDTracker, ISMRMReferenceStreamlinesTracker, StreamlinesFromFileTracker

In [ ]:
hcp_sl = CSDTracker(RLenv.dataset, random_seeds=True, seeds_count=40000)
hcp_sl.track()

In [ ]:
sls = hcp_sl.get_streamlines()

In [ ]:
streamline_index = np.random.randint(len(sls))
referenceStreamline = sls[streamline_index]

In [ ]:
filtered_streamlines = hcp_sl.filtered_streamlines_by_length(minimum=40) # filter streamlines

hcp_sl.save_to_file("data/HCP307200_CSD_min40.vtk")

In [ ]:
len(filtered_streamlines)

In [ ]:
ref = torch.FloatTensor(referenceStreamline_ijk)

In [ ]:
ref.shape

In [ ]:
pts = torch.FloatTensor(np.array([50,50,50])).view(1,-1)
pts = ref[10]+10

In [ ]:
minDist = torch.min(torch.sum( (ref - pts)**2, dim =1 ))

In [ ]:
minDist

In [ ]:
torch.nn.functional.leaky_relu(-1 * minDist)

In [ ]:
torch.where(minDist < 0.1, 1,2)